# Symmetry and Case Equivalence in Mastermind

The core of the Mastermind algorithms used here score all codewords, $AC$, against a diminishing set of remaining possibilities, $PS$, in order to subset them. The sizes of these subsets guide algorithms, such as Kinth's[1], to pick the next best guess to be played. Because there are $c^p$ codewords, this step is the most computationally intensive. Any reduction in $|AC|$ for  given $PS$ is welcomed, and indeed required for larger games.

Ville[2], section 5.4, introduces the notions of symmetry and case equivalence to provide just such a reduction. This doc expands on these ideas, and shows that we can pre-compute the sizes of these reduced $AC$ sets, $AC_r$, and share them across different $PS$. The optimized CPU solver in this repo, `SolverCPUFaster`, uses this to reduce its work or each $PS$.

These sizes are used by the CUDA solver to pack many $AC_r$ into a fixed-sized buffer, and share them across multiple $PS$, without extra intermediate space.

I will adopt Ville's terms of *free* and *zero* colors here.

[1]: D.E. Knuth. The computer as Master Mind. Journal of Recreational Mathematics, 9(1):1–6, 1976.

[2]: Geoffroy Ville, An Optimal Mastermind (4,7) Strategy and More Results in the Expected Case, March 2013, arXiv:
1305.1010 [cs.GT]. https://arxiv.org/abs/1305.1010

## Rewriting to a Class Representative

For any $p$ pin and $c$ color game, after each guess played we may have a set of *zero* colors (those which simply cannot be part of the secret), and a set of *free* colors (those which we have not yet played in this game). Given these we wish to reduce $AC$ to only those codewords which, when scored against $PS$, will provide new information and subset $PS$. For any given $PS$, there are many codewords which will provide the same outcome based on $zero$ and $free$.

A simple example is given a $4p6c$ game, and $zero={1,2}$, we can quickly see that ${1134, 1234, 2134, 2234}$ will all yield the same information. These codewords are *case equivalent*, and for any such set we would like to find its *class representative*, in this case $1134$. We need only play the class representative for each set, and thus reduce $AC$ accordingly. For zero colors, replacing all of them with the lexically first zero color produces the class representative.

Likewise, consider the same game and $free={3,4,5,6}$, which is the result of playing $1122$ initially. Again, we can see that ${3333, 4444, 5555, 6666}$ all yield the same information with regard to splitting $PS$ into further subsets. However, it is not sufficient to replace all free colors with the same one. Consider ${1556, 1565}$. The ordering of the free colors is important, and these two yield different subsets. Free colors in a codeword must be replaced, in order of appearance, with a free color in lexical order. For the codewords ${1553, 1554, 1555, 1556, 1535, 1545, 1565}$ the class representatives are ${1334, 1333, 1343}$.

Here I will use a valid codeword as the class representative for each set. Ville suggests replacing $zero$ colors with $z$ and $free$ colors, in order of appearance, with ${a,b,c,...}$. I replace $zero$ colors with the lexically first color in $zero$, and each color in $free$ with the lexically first colors in turn, in the same manner. Note carefully that all of our gameplay algorithms select the lexically first codeword when presented with multiple choices for a next best guess. It is therefore important that the class representative be the lexically first in its class.

By using a valid, lexically first codeword as the class representative we can simplify the comparison and storage costs of rewriting $AC$.





## Computing $free$ and $zero$

The free colors are easy to determine: given a history of guesses played up to the current point, which of the $c$ colors are not present in any of them?

The zero colors are also straightforward. As guesses are played we learn a number of rules about what the secret might be. If you play yourself, you might codify these in your mind as "1 cannot be in the second position, there must be a 3 in the first or fourth position, there must be a 2 or a 3, etc." However, all of these rules are implicitly encoded by $PS$, as we've removed any codeword not consistent with the guesses played at each round. Therefore, simply scanning over $PS$, we can see that any color not represented cannot be in the secret, and is thus a zero color.


## Computing $|AC_r|$

We will use the configuration of the game, $p$ and $c$, and $zero$ and $free$ computed to a certain point in gameplay. These will be the only inputs. Note that $|zero| \neq 1$ and $|free| \neq 1$, as any rewriting involving just a single zero or free color leaves the original unchanged.

The approach is to form all class representatives logically, not to rewrite the original full set of codewords, $AC$, into class representatives then sort and de-dup. By doing so we can compute the size of $AC_r$ directly.

The simplest case is to consider any game of $c>2$ colors, with two zero colors and no free colors. We only need to consider codewords without the zero colors, and we could form these $(c-2+1)^p$ codewords trivially.

Free colors are more interesting, and we will use an additive approach to construct the set. To start, consider the same game, with no zero colors and two free colors. The codewords without the free colors would be unchanged in any rewriting, so we have a baseline of $(c-2)^p$ colors to start with. We must now figure out how to form codewords using the free colors in combindation with the codewords in the baseline. The approach is to form valid sequences of free colors, of lengths $1 \dots p$, then permute those with the basline.





First consider a sequence representing $free$. The actual color values do not matter, only their number and that they have an order. We can represent them as

$$F=( a,b,c,\dotsc )$$

We will form prefix sequences of $F$, one for each possible number of free colors in a codeword for the game.

$$P_{x} =( F_{0} ,F_{1} ,\dots ,F_{x-1}),\ \forall x\in \{1,2,\dots,|free|\}$$

We will form sequences of available free colors, $G_x$, following the rule that a symbol must be proceeded by at least one occurrence of the symbol before it in $P_x$. For example, with $P_3=(a,b,c)$ then the allowable sequences are $G_3=\{(a,a,a), (a,a,b), (a,b,a), (a,b,b), (a,b,c)\}$. This constraint ensures that we replace free colors by the first observed in order, as suggested in Ville[2]. Note that we are only interested in $|G_x|$; we will form the sequences conceptually to motivate the calculation.

How many sequences can we form for each $P_x$? Consider first $P_1=(a)$, from which we can form a single sequence $G_1=\{(a)\}$.
 
Next, consider $P_2=(a,b)$. We can form $G_2=\{(a,a), (a,b)\}$ by appending each symbol from $P_2$ to each sequence in $G_1$. We can do this and obey the constraint above because the only symbol in $(a)$ can be followed by any symbol in $P_2$.

Now, what sequences can we form with $P_3=(a,b,c)$? Building upon $G_2$, we can see that we will be appending individual symbols from $P_3$. $P_2$ produced one sequence which only contains the first symbol, and one sequence which contained both. This first sequence $(a,a)$ can only support the addition of the first two symbols in $P_3$, i.e., $P_2$, while the last sequence $(a,b)$ can support the addition of all symbols in $P_3$. Thus, we can form $G_3=\{(a,a,a), (a,a,b), (a,b,a), (a,b,b), (a,b,c)\}$, the first two built from $(a,a)$ and the next three built from $(a,b)$.

Looking at $G_3$, again we see the first sequence can only support the first two symbols from $P_4$, i.e., $P_2$, and the last sequence can support any symbol from $P_4$. Of the rest, some support just the first three symbols of $P_4$, i.e., $P_3$, while others support all four, and the difference is determined by the sequence in $G_2$ from which they were formed. This pattern continues, with the formable sequences being dictated by the origin of their predecessors.

We may visualize this with a table. Cells in the table represent how many sequences in $G_{n}$ can be combined with each $P_x$ to form $G_{n+1}$, and rows add up to the total number of sequences in each $G_n$. 

|       | $P_1$ | $P_2$  | $P_3$  | $P_4$  | $P_5$  |
|:------|:------|:------:|:------:|:------:|:------:|
| $G_1$ |       |   1    |        |        |        |
| $G_2$ |       |   1    |   1    |        |        |
| $G_3$ |       |   1    |   3    |   1    |        |
| $G_4$ |       |   1    |   7    |   6    |   1    |

This pattern continues forward and is expressed by

$$f(n,x) ::= \begin{cases}
1 & \text{if } n=1,x=2\\
f(n-1,x-1)+(|P_{x}|-1)f(n-1,x) & \text{if } |P_{x}|<|F|\\
f(n-1,x-1)+|P_{x}|f(n-1,x) & \text{if } |P_{x}|=|F|\\
0 & \text{otherwise}
\end{cases}$$

$$|G_{n}|=\sum_{x=2}^{n+1}f(n,x)$$


In [1]:
def generated_counts(p, free):
    f = [[0 for i in range(16)] for i in range(16)]
    g = [0] * 16
    f[1][2] = 1
    g[1] = 1
    for n in range(2, p + 1):
        for x in range(2, len(free) + 1):
            # nb: |P_x| == x
            if x < len(free):
                f[n][x] = f[n - 1][x - 1] + (x - 1) * f[n - 1][x]
            else:
                f[n][x] = f[n - 1][x - 1] + x * f[n - 1][x]
            g[n] += f[n][x]
    return g


assert (generated_counts(6, (1, 2, 3, 4, 5, 6, 7))[6] == 203)
assert (generated_counts(6, (1, 2))[6] == 32)
assert (generated_counts(6, (1, 2, 3))[6] == 122)
generated_counts(4, (1, 2, 3, 4))

[0, 1, 2, 5, 15, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

We will use $G_n$ to form new codewords by combining its sequences with the available colors in $PS$. For any given sequence of $n$ colors, how many length $p$ codewords can be formed by filling $p-n$ gaps with other colors from $PS$? This is given by the binomial coefficient

$$C(p,n)=\frac{p!}{n! (p-n)!}.\ \forall n\leqslant p$$

In [2]:
import math


def combos(p, n):
    return math.factorial(p) / (math.factorial(n) * math.factorial(p - n))

The number of sequences of each length is
$$S_{n}=C(p,n)G_n.\ n\in \{1,\cdots,p\}$$

In [3]:
def seqs_for_length(p, length, G):
    return combos(p, length) * G[length]

Next we need to know how many codewords $C_a$ can be formed from the available colors in $PS$. These are the colors which are not in $zero$ and not in $free$.
 
$$C_{a}=\begin{cases}
c-|free|-|zero|+1 & \text{if } |zero|>0\\
max(1,c-|free|) & \text{otherwise}
\end{cases}
$$

In [4]:
def available_colors(c, free, zero):
    if len(zero) > 0:
        return c - len(free) - len(zero) + 1
    return max(1, c - len(free))

We can now combine the available colors with the sequences of each length, and the number of total sequences formed from the free sequences is therefore
$$FS=\sum_{n=1}^{p} S_n C_{a}^{p-n}$$


In [5]:
def free_seqs(p, c, free, zero):
    fs = 0
    G = generated_counts(p, free)
    for n in range(1, p + 1):
        fs += seqs_for_length(p, n, G) * available_colors(c, free, zero) ** (p - n)
    return int(fs)


assert (free_seqs(4, 12, (1, 2), ()) == 5368)

Finally, the total number of codewords in the reduced set is

$$|AC_r|=\begin{cases}
C_{a}^{p}+FS & \text{if } |Free|>0\\
(c-|zero|+1)^{p} & 
\end{cases}$$

The specific colors in $free$ and $zero$ don't matter, only $|free|$ and $|zero|$ matters.


In [6]:
def size_acr(p, c, free, zero):
    if len(free) > 0:
        return available_colors(c, free, zero) ** p + free_seqs(p, c, free, zero)
    return (c - len(zero) + 1) ** p


assert (size_acr(4, 12, (1, 2), ()) == 15368)
assert (size_acr(4, 12, (1, 2, 3, 4), ()) == 7087)
assert (size_acr(4, 12, (1, 2, 3, 4), (1, 2, 3, 4, 5, 6, 7, 8)) == 52)
assert (size_acr(4, 12, (), (1, 2)) == 14641)
assert (size_acr(4, 12, (1, 2), (1, 2)) == 10601)
size_acr(4, 6, (5, 6), (1, 2))

353

Let's compute $|AC_r|$ for all games we'll play for now, and for all potential combination of $zero$ and $free$ within each game, and save that to a JSON file we can use while playing.

In [7]:
import json

ACr = {}
for p in range(2, 8 + 1):
    for c in range(2, 15 + 1):
        for z in range(0, c + 1):
            if z == 1:
                continue
            for f in range(0, c + 1):
                if f == 1:
                    continue
                if z + f > c:
                    continue
                if z + f == 0:
                    continue
                key = f"{p:02d}{c:02d}{z:02d}{f:02d}"
                ACr[key] = size_acr(p, c, [1] * f, [1] * z)

with open('../ACReduced.json', 'w') as f:
    json.dump(ACr, f)

len(ACr)

4018

In [8]:
from itertools import islice

list(islice(ACr.items(), 10))

[('02020002', 5),
 ('02020200', 1),
 ('02030002', 5),
 ('02030003', 5),
 ('02030200', 4),
 ('02030300', 1),
 ('02040002', 10),
 ('02040003', 5),
 ('02040004', 5),
 ('02040200', 9)]